In [1]:
import tensorflow

print(tensorflow.__version__)

2.6.0


In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
#샘플 데이터 불러옴
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
144364,Would you like to come to my party?,Voudrais-tu venir à ma fête ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
15411,My wife was mad.,Ma femme était folle.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
188116,"In a town with only one barber, who shaves the...",Dans une ville avec un seul coiffeur pour homm...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
138977,Where have you been all afternoon?,Où as-tu été toute l'après-midi ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
142597,Save it on the external hard drive.,Enregistre-le sur le disque dur externe.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [4]:
# 3번째 열은 불필요하므로 제거하고, 훈련 데이터는 33000개의 샘플로 줄임
lines = lines[['eng', 'fra']][:33000] # 33000개 샘플 사용 3000개는 추후 테스트 샘플로 사용
lines.sample(5)

,eng,fra
15005,It looked awful.,Ça avait l'air atroce.
3420,I'm finicky.,Je suis méticuleuse.
2,Go.,En route !
26151,The tank is empty.,La cuve est vide.
31857,That was very good.,Ce fut très bon.


In [5]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
20187,Please stay here.,\t Veuillez rester ici. \n
26,Duck!,\t À terre ! \n
13198,Behave yourself.,\t Comporte-toi bien. \n
3,Go.,\t Bouge ! \n
6227,Try it again.,\t Essaye à nouveau. \n


In [6]:
# 각 데이터 프레임 칼럼을 리스트화
eng_list = lines['eng'].to_list()
fra_list = lines['fra'].to_list()

In [7]:
eng_list[:5]

['Go.', 'Go.', 'Go.', 'Go.', 'Hi.']

In [8]:
# 1. 구두점을 단어와 분리
import re, string

def punc(s): return re.sub(f"([{string.punctuation}])", r' \1 ', s)

In [9]:
p_el = [punc(i) for i in eng_list]

In [10]:
p_el[:5]

['Go . ', 'Go . ', 'Go . ', 'Go . ', 'Hi . ']

In [11]:
p_fl = [punc(i) for i in fra_list]

In [12]:
p_fl[:5]

['\t Va  !  \n',
 '\t Marche .  \n',
 '\t En route  !  \n',
 '\t Bouge  !  \n',
 '\t Salut  !  \n']

In [13]:
# 소문자로 바꾸고 띄어쓰기 단위로 토큰화 
eng_tokenizer = Tokenizer(char_level=False, filters='', lower =True)   # 단어 단위로 Tokenizer를 생성합니다. 소문자로 바꾸고 필터(구두점제거)
eng_tokenizer.fit_on_texts(p_el)               # 33000개의 행을 가진 p_el의 각 요소들에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(p_el)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[29, 1], [29, 1], [29, 1]]

In [14]:
fra_tokenizer = Tokenizer(char_level=False, filters='', lower =True)   # 단어 단위로 Tokenizer를 생성합니다. 소문자로 바꾸고 필터(구두점제거)
fra_tokenizer.fit_on_texts(p_fl)               # 33000개의 행을 가진 p_fl의 각 요소들에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(p_fl)    # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 76, 10, 2], [1, 345, 3, 2], [1, 27, 486, 10, 2]]

In [15]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4713
프랑스어 단어장의 크기 : 9467


In [16]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 20


In [17]:
#전체적인 통계정보를 한꺼번에 출력

print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4713
프랑스어 단어장의 크기 : 9467
영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 20


In [18]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [19]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 76, 10], [1, 345, 3], [1, 27, 486, 10]]
[[76, 10, 2], [345, 3, 2], [27, 486, 10, 2]]


In [20]:
#데이터의 크기 확인
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10)
프랑스어 입력데이터의 크기(shape) : (33000, 20)
프랑스어 출력데이터의 크기(shape) : (33000, 20)


In [21]:
print(encoder_input[0])

[29  1  0  0  0  0  0  0  0  0]


In [22]:
# 벡터화 원핫 인코딩
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10, 4713)
프랑스어 입력데이터의 크기(shape) : (33000, 20, 9467)
프랑스어 출력데이터의 크기(shape) : (33000, 20, 9467)


In [23]:
# validation 생성 3000건 나머지 학습데이터로
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 10, 4713)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 20, 9467)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 20, 9467)


In [24]:
#프랑수아 숄레의 케라스의 seq2seq 구현 가이드 게시물인 A ten-minute introduction to sequence-to-sequence learning를 참고
#도구 임포트

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model


In [25]:
#임베딩 층 구현
from tensorflow.keras.layers import Input, Embedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시(영어) 임베딩 벡터의 차원(word_vec_dim)을 16으로 설정 하이퍼 파라미터
word_vec_dim = 8
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(eng_vocab_size, word_vec_dim )(encoder_inputs)
# hidden size가 16인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 16, return_state=True)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [26]:
#디코더 설계
# 입력 텐서 생성. word_vec_dim 16 동일

decoder_inputs = Input(shape=(None,))
dec_emb =  Embedding(fra_vocab_size, word_vec_dim )(encoder_inputs)
# hidden size가 16인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 16, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(dec_emb, initial_state = encoder_states)

In [27]:
#디코더의 출력층 설계
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [28]:
#매 time step마다의 다중 클래스 분류 문제이므로 프랑스어 단어장으로부터 한 가지 문자만 선택하도록 합니다. Dense의 인자로 프랑스어 단어장의 크기를 기재하고, 활성화 함수로 소프트맥스 함수를 사용. 최종적으로 인코더와 디코더를 연결해서 하나의 모델로 만들어줍니다. Model의 Input과 Output의 정의를 유심히 살펴 주세요.

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 8)      37704       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 8)      75736       input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 16), (None,  1600        embedding[0][0]                  
______________________________________________________________________________________________

In [ ]:
history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
                    validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
                    batch_size=16, epochs=15)

# 회고

- 지난 ex 6에서는 직접 구두점들을 설정하고 다루는 코드를 작성하여 사용하였으나,
- 이번에는 string 내장 함수를 사용하여 구두점 작업을 하였다.
- sentence to sentence 의 번역기 코드를 작성해 볼 수 있었다.
- 직접 임베딩 레이어를 추가하여 단어의 임베딩 깊이를 조절해 볼 수 있었다.
- 영어를 인코딩 하여 어떤 x로 바꾸고 그 x를 디코딩하여 프랑스어로 바꾸는 알고리즘으로 러닝 모델을 작성해 보았다. 
- Total params를 최대한으로 줄여서 모델 학습을 시켜보려고 하였으나 클라우드 서버가 계속 다운이 되어 진행시키지 못했다.
- 코드 자체의 문제인지 많은사람이 한꺼번에 코드를 돌려서 서버 리소스의 문제인지 확인을 하지 못하였다.
- 지난번 데이터톤 때에도 느꼈지만 장비(GPU TPU등 연산장치 또는 머신러닝을 구현하는데 필요한 부수 장비)의 성능 및 발전도 머신러닝을 연구하고 발전시키는데 아주 중요한 부분임을 새삼 깨달았다
- colab또는 좋은 머신러닝구현 환경을 사용하도록 준비를 해야겠다.